# Reversing Tips

Five tips to level up your reverse engineering with Python3.

## Tip 1 - Use Juptyper Notebooks and A Git Repo

Remember to steal our .gitignore file to filter out the juptyper junk files.

## Tip 2 - Remember Byte Strings Are Not Strings

In [1]:
import binascii

string_example = "test"
byte_array_example = b"test"

# Convert string into bytes
print(string_example.encode('utf-8'))

# Convert byte array into string
print(byte_array_example.decode('utf-8'))

b'test'
test


## Example for Tips 3-5

In [9]:
example_data = b'\x02\x00\x00\x00\x00\x04\x00\x00\x00test\x01\x04\x00\x00\x00t\x00e\x00s\x00t\x00'

### Example Data Structure

```
struct strings{
DWORD number_of_strings;
string* string;
}

struct string{
BOOL is_wide_string;
DWORD string_length;
chr* string;
}
```



## Tip 3 - Hex Encoding Binary Data

In [5]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

    
def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

print("This is hex encoded data: %r" % tohex(example_data))

This is hex encoded data: b'0200000000040000007465737401040000007400650073007400'


## Tip 4 - Use Struct To Convert Between Data and Types

https://docs.python.org/3/library/struct.html


In [11]:
import struct
number_of_strings = struct.unpack('<I',example_data[:4])[0]
print("Number of strings: %d" % number_of_strings)

Number of strings: 2


## Tip 5 - Use Custom Struct Class To Parse Binary Data

In [24]:
import struct 

example_string = b'\x00\x04\x00\x00\x00test'

class EXAMPLE_STRING:
    def __init__(self):
        self.is_wide_string = False
        self.string_length = 0
        self.string = b''
    def from_buffer_copy(self, data):
        ptr = 0
        self.is_wide_string = struct.unpack('?', data[ptr:ptr+1])[0]
        ptr += 1
        self.string_length = struct.unpack('<I', data[ptr:ptr+4])[0]
        ptr += 4
        if self.is_wide_string:
            self.string = data[ptr:ptr+(self.string_length*2)].decode('utf-16le')
            ptr += self.string_length*2
        else:
            self.string = data[ptr:ptr+self.string_length].decode('utf-8')
            ptr += self.string_length
    def pack(self):
        data = b''
        data += struct.pack('?', self.is_wide_string)
        data += struct.pack('<I', self.string_length)
        if self.is_wide_string:
            data += self.string.encode('utf-16le')
        else:
            data += self.string.encode('utf-8')
        return data

print("Example string data: %r" % example_string)
es = EXAMPLE_STRING()
es.from_buffer_copy(example_string)
print("Example is wide: %s" % es.is_wide_string)
print("Example string length: %d" % es.string_length)
print("Example string: %s" % es.string)
        

Example string data: b'\x00\x04\x00\x00\x00test'
Example is wide: False
Example string length: 4
Example string: test


In [25]:
es.is_wide_string = True
print("Example string data converted to wide: %r" % es.pack())

Example string data converted to wide: b'\x01\x04\x00\x00\x00t\x00e\x00s\x00t\x00'


In [30]:

class EXAMPLE_STRINGS:
    def __init__(self):
        self.length = 0
        self.strings = []
    def from_buffer_copy(self, data):
        ptr = 0
        self.length = struct.unpack('<I', data[ptr:ptr+4])[0]
        ptr += 4
        for i in range(self.length):
            tmp_string = EXAMPLE_STRING()
            tmp_string.from_buffer_copy(data[ptr:])
            ptr += len(tmp_string.pack())
            self.strings.append(tmp_string)
    def pack(self):
        data = b''
        data += struct.pack('<I', self.length)
        for s in self.strings:
            data += s.pack()
        return data
    
es = EXAMPLE_STRINGS()
es.from_buffer_copy(example_data)
print("Number of strings: %d" % es.length)
print("First string is wide: %s" % es.strings[0].is_wide_string)
print("First string: %s" % es.strings[0].string)
print("Second string is wide: %s" % es.strings[1].is_wide_string)
print("Second string: %s" % es.strings[1].string)

Number of strings: 2
First string is wide: False
First string: test
Second string is wide: True
Second string: test
